In [1]:
ENV["COLUMNS"] = 120

using HypothesisTests

# minlik版のFisher検定の実装では ≤ ではなく, 次の ⪅ を使う必要がある.
x ⪅ y = x < y || x ≈ y

# minlik版Fisher検定のP値の素朴だが(高率の悪いが)正しい実装
using Distributions
function fisher_test(a, b, c, d)
    hg = Hypergeometric(a+b, c+d, a+c)
    pa = pdf(hg, a)
    sum(pdf(hg, i) for i in support(hg) if pdf(hg, i) ⪅ pa)
end

# ≤ を使う正しくない実装
function fisher_test_le(a, b, c, d)
    hg = Hypergeometric(a+b, c+d, a+c)
    pa = pdf(hg, a)
    sum(pdf(hg, i) for i in support(hg) if pdf(hg, i) ≤ pa)
end

# https://qiita.com/WolfMoon/items/530413ce7c8439d18c18
using SpecialFunctions
# 以下も不適切な実装
function fisher_test_wm(a, b, c, d)
    lchoose(n, k) = logfactorial(n) - logfactorial(k) - logfactorial(n - k)
    Stats(i, e, f, g, n) = exp(lchoose(e, i) + lchoose(f, g - i) - lchoose(n, g))
    e, f, g, h, n = a + b, c + d, a + c, b + d, a + b + c + d
    mi = max(0, e + g - n)
    length = min(e, g) - mi
    prob = [Stats(mi + i, e, f, g, n) for i in 0:length]
    #println("Fisher's Exact Test for Count Data (two tailed)")
    #println("p value =", sum(prob[prob .<= Stats(a, e, f, g, n)]))
    sum(prob[prob .<= Stats(a, e, f, g, n)])
end

# 訂正版
function fisher_test_rev(a, b, c, d)
    lchoose(n, k) = logfactorial(n) - logfactorial(k) - logfactorial(n - k)
    Stats(i, e, f, g, n) = exp(lchoose(e, i) + lchoose(f, g - i) - lchoose(n, g))
    e, f, g, h, n = a + b, c + d, a + c, b + d, a + b + c + d
    mi = max(0, e + g - n)
    length = min(e, g) - mi
    prob = [Stats(mi + i, e, f, g, n) for i in 0:length]
    #println("Fisher's Exact Test for Count Data (two tailed)")
    #println("p value =", sum(prob[prob .<= Stats(a, e, f, g, n)]))
    sum(prob[prob .⪅ Stats(a, e, f, g, n)])
end

fisher_test_rev (generic function with 1 method)

In [2]:
s, t, n = 17, 7, 12
hg = Hypergeometric(s, t, n)
supp = support(hg)
Any[
    [(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_wm(a, s-a, n-a, t-n+a) for a in supp];;
    [pvalue(FisherExactTest(a, s-a, n-a, t-n+a), method=:minlike) for a in supp];;
    [fisher_test(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_le(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_wm(a, s-a, n-a, t-n+a) ≈ pvalue(FisherExactTest(a, s-a, n-a, t-n+a), method=:minlike) for a in supp];;
    [fisher_test_wm(a, s-a, n-a, t-n+a) ≈ fisher_test(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test(a, s-a, n-a, t-n+a) ≈ fisher_test_le(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_rev(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_rev(a, s-a, n-a, t-n+a) ≈ fisher_test(a, s-a, n-a, t-n+a) for a in supp];;
]

8×10 Matrix{Any}:
 (5, 12, 7, 0)  0.00457666  0.00457666  0.00457666  0.00457666   true   true   true  0.00457666  true
 (6, 11, 6, 1)  0.0686499   0.0686499   0.0686499   0.0366133    true   true  false  0.0686499   true
 (7, 10, 5, 2)  0.370709    0.370709    0.370709    0.370709     true   true   true  0.370709    true
 (8, 9, 4, 3)   1.0         1.0         1.0         1.0          true   true   true  1.0         true
 (9, 8, 3, 4)   0.685355    1.0         1.0         0.685355    false  false  false  1.0         true
 (10, 7, 2, 5)  0.370709    0.370709    0.370709    0.370709     true   true   true  0.370709    true
 (11, 6, 1, 6)  0.0366133   0.0686499   0.0686499   0.0686499   false  false   true  0.0686499   true
 (12, 5, 0, 7)  0.00457666  0.00457666  0.00457666  0.00457666   true   true   true  0.00457666  true

In [3]:
a, b, c, d = 11, 6, 1, 6
@show fisher_test_wm(a, b, c, d)
@show pvalue(FisherExactTest(a, b, c, d))
@show fisher_test(a, b, c, d)
@show fisher_test_le(a, b, c, d);

fisher_test_wm(a, b, c, d) = 0.036613272311213224
pvalue(FisherExactTest(a, b, c, d)) = 0.068649885583524
fisher_test(a, b, c, d) = 0.06864988558352397
fisher_test_le(a, b, c, d) = 0.06864988558352397


In [4]:
a, b, c, d = 9, 8, 3, 4
@show fisher_test_wm(a, b, c, d)
@show pvalue(FisherExactTest(a, b, c, d))
@show fisher_test(a, b, c, d)
@show fisher_test_le(a, b, c, d);

fisher_test_wm(a, b, c, d) = 0.6853546910755224
pvalue(FisherExactTest(a, b, c, d)) = 0.9999999999999998
fisher_test(a, b, c, d) = 0.9999999999999999
fisher_test_le(a, b, c, d) = 0.6853546910755147


In [5]:
s, t, n = 17, 7, 12
hg = Hypergeometric(s, t, n)
supp = support(hg)
[(a, pdf(Hypergeometric(s, t, n), a)) for a in supp]

8-element Vector{Tuple{Int64, Float64}}:
 (5, 0.0022883295194507988)
 (6, 0.032036613272311186)
 (7, 0.1510297482837528)
 (8, 0.31464530892448517)
 (9, 0.3146453089244851)
 (10, 0.1510297482837528)
 (11, 0.0320366132723112)
 (12, 0.0022883295194507988)

In [6]:
using HypothesisTests
using RCall
using SpecialFunctions

# https://qiita.com/WolfMoon/items/530413ce7c8439d18c18
function fisher(a, b, c, d)
    lchoose(n, k) = logfactorial(n) - logfactorial(k) - logfactorial(n - k)
    Stats(i, e, f, g, n) = exp(lchoose(e, i) + lchoose(f, g - i) - lchoose(n, g))
    e, f, g, h, n = a + b, c + d, a + c, b + d, a + b + c + d
    mi = max(0, e + g - n)
    length = min(e, g) - mi
    prob = [Stats(mi + i, e, f, g, n) for i in 0:length]
    println("Fisher's Exact Test for Count Data (two tailed)")
    println("p value =", sum(prob[prob .<= Stats(a, e, f, g, n)]))
end;

In [7]:
fisher(11, 6, 1, 6)

Fisher's Exact Test for Count Data (two tailed)
p value =0.036613272311213224


In [8]:
pvalue(FisherExactTest(11, 6, 1, 6), method=:minlike)

0.06864988558352403

In [9]:
R"fisher.test(matrix(c(11, 6, 1, 6), nrow=2))"

RObject{VecSxp}

	Fisher's Exact Test for Count Data

data:  matrix(c(11, 6, 1, 6), nrow = 2)
p-value = 0.06865
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
   0.8868833 548.6839949
sample estimates:
odds ratio 
  9.910647 



In [10]:
fisher(9, 8, 3, 4)

Fisher's Exact Test for Count Data (two tailed)
p value =0.6853546910755224


In [11]:
pvalue(FisherExactTest(9, 8, 3, 4), method=:minlike)

1.0

In [12]:
R"fisher.test(matrix(c(9, 8, 3, 4), nrow=2))"

RObject{VecSxp}

	Fisher's Exact Test for Count Data

data:  matrix(c(9, 8, 3, 4), nrow = 2)
p-value = 1
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
  0.1835411 13.3708178
sample estimates:
odds ratio 
  1.474776 

